In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv("Data/titanic/train.csv")

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
columns_to_drop= ["PassengerId", "Name", "Ticket", "Cabin", "Embarked"]

In [6]:
data_clean= data.drop(columns_to_drop, axis=1)

In [7]:
data_clean.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [8]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
data_clean["Sex"]= le.fit_transform(data_clean["Sex"])

In [9]:
data_clean.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [10]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int32  
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int32(1), int64(4)
memory usage: 45.4 KB


In [11]:
data_clean=data_clean.fillna(data_clean["Age"].mean())

In [12]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int32  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int32(1), int64(4)
memory usage: 45.4 KB


In [13]:
input_cols=['Pclass', "Sex", "Age", "SibSp", "Parch", "Fare"]
output_cols=["Survived"]
x=data_clean[input_cols]
y=data_clean[output_cols]
print(x.shape,y.shape)
print(type(x))

(891, 6) (891, 1)
<class 'pandas.core.frame.DataFrame'>


In [14]:
def entropy(col):
    counts=np.unique(col, return_counts=True)
    n=float(col.shape[0])
    ent=0.0
    for ix in counts[1]:
        p=ix/n
        ent= ent+(-1.0*p*np.log2(p))
        
    return ent

In [15]:
def divide_data(x_data,fkey,fval):
    x_right= pd.DataFrame([],columns=x_data.columns)
    x_left=pd.DataFrame([],columns=x_data.columns)
    
    for ix in range (x_data.shape[0]):
        val=x_data[fkey].loc[ix]
        
        if val>fval:
            x_right=x_right.append(x_data.loc[ix])
        else:
            x_left=x_left.append(x_data.loc[ix])
    return x_left,x_right

In [16]:
def information_gain(x_data,fkey,fval):
    left,right= divide_data(x_data, fkey, fval)
    l=float(left.shape[0])/x_data.shape[0]
    r=float(right.shape[0])/x_data.shape[0]
    
    if left.shape[0] == 0 or right.shape[0] ==0:
        return -1000000
    
    i_gain= entropy(x_data.Survived)-(l*entropy(left.Survived)+r*entropy(right.Survived))
    return i_gain

In [17]:
for fx in x.columns:
    print(fx)
    print(information_gain(data_clean, fx, data_clean[fx].mean()))

Pclass
0.07579362743608165
Sex
0.2176601066606142
Age
0.001158644038169343
SibSp
0.009584541813400071
Parch
0.015380754493137694
Fare
0.042140692838995464


In [18]:
class DecisionTree:
    
    
    def __init__(self,depth=0,max_depth=5):
        self.left=None
        self.right=None
        self.fkey=None
        self.fval=None
        self.max_depth=max_depth
        self.depth=depth
        self.target=None
        
    def train(self, x_train):
        features=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
        info_gains=[]
        
        for ix in features:
            i_gain=information_gain(x_train, ix, x_train[ix].mean())
            info_gains.append(i_gain)
            
        self.fkey=features[np.argmax(info_gains)]
        self.fval=x_train[self.fkey].mean()
        
        print("making tree features is", self.fkey)
        
        #split data
        data_left, data_right= divide_data(x_train, self.fkey, self.fval)
        data_left= data_left.reset_index(drop=True)
        data_right=data_right.reset_index(drop=True)
        
        #truly a leaf node
        if data_left.shape[0] == 0 or data_right.shape[0] == 0:
            if x_train.Survived.mean() >= 0.5:
                self.target="Survive"
            else:
                self.target="Dead"
            return
        
        #stop early when depth>=max depth
        if(self.depth>=self.max_depth):
            if x_train.Survived.mean() >= 0.5:
                self.target="Survive"
            else:
                self.target="Dead"
            return
        
        #recursive case
        self.left=DecisionTree(depth=self.depth+1, max_depth=self.max_depth)
        self.left.train(data_left)
        
        self.right=DecisionTree(depth=self.depth+1, max_depth=self.max_depth)
        self.right.train(data_right)
        
        #you can set target at every node
        if x_train.Survived.mean() >= 0.5:
            self.target="Survive"
        else:
            self.target="Dead"
        return
    
    def predict(self,test):
        if test[self.fkey]>self.fval:
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)
    
            
        

In [19]:
d= DecisionTree()
d.train(data_clean)

making tree features is Sex
making tree features is Pclass
making tree features is Pclass
making tree features is Parch
making tree features is Age
making tree features is Age
making tree features is Age
making tree features is Parch
making tree features is Age
making tree features is Fare
making tree features is Parch
making tree features is Age
making tree features is Age
making tree features is Age
making tree features is Age
making tree features is Age
making tree features is Age
making tree features is Fare
making tree features is SibSp
making tree features is Fare
making tree features is Fare
making tree features is Age
making tree features is SibSp
making tree features is Parch
making tree features is Age
making tree features is SibSp
making tree features is Fare
making tree features is Parch
making tree features is Parch
making tree features is Age
making tree features is Age
making tree features is Age
making tree features is Fare
making tree features is Parch
making tree feat

In [20]:
split=int(0.7*data_clean.shape[0])
train_data= data_clean[:split]
test_data= data_clean[split:]
test_data=test_data.reset_index(drop=True)

In [21]:
print(train_data.shape, test_data.shape)

(623, 7) (268, 7)


In [23]:
print(d.fkey)
print(d.fval)
print(d.left.fkey)
print(d.right.fkey)

Sex
0.6475869809203143
Pclass
Fare


In [24]:
y_pred=[]
for ix in range(test_data.shape[0]):
    y_pred.append(d.predict(test_data.loc[ix]))

In [25]:
y_pred

['Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 

In [26]:
y_actual=test_data[output_cols]
print(y_actual)

     Survived
0           0
1           0
2           0
3           0
4           1
..        ...
263         0
264         1
265         0
266         1
267         0

[268 rows x 1 columns]


In [27]:
le=LabelEncoder()
y_pred=le.fit_transform(y_pred)
print(y_pred)

[0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0
 1 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 1 0 0 1 1 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0
 0 1 0 1 0 0 1 0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 1 0 0 0 1
 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0
 0 0 0 0 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 0 1 0 0 1 1 0 0 0 1 1 0
 1 0 0 0 0 1 1 1 0]


In [28]:
y_pred=np.array(y_pred).reshape((-1,1))
print(y_pred.shape)

(268, 1)


In [29]:
acc=np.sum(y_pred==y_actual)/y_pred.shape[0]

In [30]:
y_pred==y_actual

,Survived
0,True
1,True
2,True
3,True
4,True
...,...
263,True
264,True
265,False
266,True


In [31]:
acc=np.sum(np.array(y_pred)==np.array(y_actual))/y_pred.shape[0]

In [32]:
print(acc)

0.8619402985074627
